In [1]:
import pandas as pd
from datetime import datetime

## Dataset Analysis
put some description here ig

In [6]:
ditem = pd.read_csv('data/MIMIC-III/D_ITEMS.csv')
labitem = pd.read_csv('data/MIMIC-III/D_LABITEMS.csv')
chart_df = pd.read_csv('./data/MIMIC-III/target_population/filtered_chartevents_1000.csv')
lab_df = pd.read_csv('data/MIMIC-III/LABEVENTS.csv')
icu_sample = pd.read_csv('data/MIMIC-III/target_population/sample_icu.csv')

/tmp/ipykernel_11428/3002774615.py:3: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  chart_df = pd.read_csv('./data/MIMIC-III/target_population/filtered_chartevents_1000.csv')


In [4]:
inputcv = pd.read_csv('data/MIMIC-III/INPUTEVENTS_CV.csv')
inputmv = pd.read_csv('data/MIMIC-III/INPUTEVENTS_MV.csv')
output_df = pd.read_csv('data/MIMIC-III/OUTPUTEVENTS.csv')
procedures = pd.read_csv('data/MIMIC-III/PROCEDUREEVENTS_MV.csv')

/tmp/ipykernel_11428/362509180.py:1: DtypeWarning: Columns (7,9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  inputcv = pd.read_csv('data/MIMIC-III/INPUTEVENTS_CV.csv')


In [25]:
# filter all structured data files so only HADM_ID that are in sample cohort (1000)
inputcv = inputcv[inputcv['HADM_ID'].isin(icu_sample['HADM_ID'])].reset_index(drop=True)
inputmv = inputmv[inputmv['HADM_ID'].isin(icu_sample['HADM_ID'])].reset_index(drop=True)
output_df = output_df[output_df['HADM_ID'].isin(icu_sample['HADM_ID'])].reset_index(drop=True)
procedures = procedures[procedures['HADM_ID'].isin(icu_sample['HADM_ID'])].reset_index(drop=True)
chart_df = chart_df[chart_df['HADM_ID'].isin(icu_sample['HADM_ID'])].reset_index(drop=True)
lab_df = lab_df[lab_df['HADM_ID'].isin(icu_sample['HADM_ID'])].reset_index(drop=True)

In [28]:
def get_duration(start: str, end: str):
    format_str = r'%Y-%m-%d %H:%M:%S'
    dt_start = datetime.strptime(start, format_str)
    dt_end = datetime.strptime(end, format_str)
    duration = dt_end - dt_start
    duration_minutes = int(duration.total_seconds()) / 60
    return duration_minutes

In [29]:
inputmv['DURATION'] = inputmv.apply(lambda x: get_duration(x['STARTTIME'], x['ENDTIME']), axis=1)
col = inputmv.pop('DURATION')
inputmv.insert(6, 'DURATION', col)
inputmv.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,DURATION,ITEMID,AMOUNT,AMOUNTUOM,...,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
0,241,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,136.0,225166,6.774532,mEq,...,ml,0,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10.000000,0.050000
1,242,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,56.0,225944,28.132997,ml,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,28.132998,30.255817
2,243,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,56.0,225166,2.813300,mEq,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426
3,244,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-03 12:01:00,1.0,225893,1.000000,dose,...,ml,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,1.000000,1.000000
4,245,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-03 12:01:00,1.0,220949,100.000000,ml,...,ml,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,100.000000,0.000000


In [30]:
# error in the data, time measured in hours but should be minute (compare start and end timestamps for reference)

procedures['VALUEUOM'] = procedures['VALUEUOM'].replace('minute', 'min')
procedures.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,...,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,22,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-06 18:09:00,224264,4689.0,min,NaN,...,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
1,23,27063,139787,223259.0,2133-02-03 15:21:00,2133-02-03 15:22:00,225447,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,1,Rewritten,NaN,RN,2133-02-03 15:21:00
2,24,27063,139787,223259.0,2133-02-03 15:22:00,2133-02-03 15:23:00,225447,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
3,25,27063,139787,223259.0,2133-02-03 15:22:00,2133-02-03 15:23:00,225479,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
4,26,27063,139787,223259.0,2133-02-04 08:54:00,2133-02-04 08:55:00,225402,1.0,NaN,NaN,...,Procedures,NaN,Electrolytes,0,0,1,Rewritten,NaN,RN,2133-02-04 08:54:00


In [32]:
lab_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,76887,137,151583.0,50809,2191-11-16 10:15:00,175,175.0,mg/dL,abnormal
1,76888,137,151583.0,50810,2191-11-16 10:15:00,30,30.0,%,NaN
2,76889,137,151583.0,50811,2191-11-16 10:15:00,9.9,9.9,g/dL,abnormal
3,76890,137,151583.0,50813,2191-11-16 10:15:00,0.9,0.9,mmol/L,NaN
4,76891,137,151583.0,50818,2191-11-16 10:15:00,48,48.0,mm Hg,abnormal


In [50]:
# Group by 'ITEMID'
# grouped = lab_df.groupby('ITEMID')
# grouped = chart_df.groupby('ITEMID')
# grouped = inputcv.groupby('ITEMID')
# grouped = inputmv.groupby('ITEMID')
# grouped = output_df.groupby('ITEMID')
grouped = procedures.groupby('ITEMID')

item_ids = grouped.size().index.values  # Group keys (unique ITEMIDs)

# Merge to get labels from ditem based on ITEMIDs
# item_labels = pd.merge(pd.DataFrame(item_ids, columns=['ITEMID']), labitem[['ITEMID', 'LABEL']], on='ITEMID', how='left')['LABEL']
item_labels = pd.merge(pd.DataFrame(item_ids, columns=['ITEMID']), ditem[['ITEMID', 'LABEL']], on='ITEMID', how='left')['LABEL']

# Efficient statistics calculation
uoms = grouped['VALUEUOM'].first().values
averages = grouped['VALUE'].mean().values
mins = grouped['VALUE'].min().values
maxs = grouped['VALUE'].max().values
vals = grouped.size().values
# rate = grouped['RATE'].mean().values
# rateuom = grouped['RATEUOM'].first().values
# ave_dur = grouped['DURATION'].mean().values

In [51]:
# Construct a DataFrame with the computed columns
result_df = pd.DataFrame({
    'ITEMID': item_ids,
    'LABEL': item_labels,
    'AMOUNT_UOM': uoms,
    'AVG_AMOUNT': averages,
    'MIN': mins,
    'MAX': maxs,
    'COUNT': vals,
    # 'RATE_AVG': rate,
    # 'RATE_UOM': rateuom,
    # 'AVG_DURATION': ave_dur
})

# Show the resulting DataFrame
result_df

,ITEMID,LABEL,AMOUNT_UOM,AVG_AMOUNT,MIN,MAX,COUNT
0,221214,CT scan,None,1.0,1.0,1.0,300
1,221216,X-ray,None,1.0,1.0,1.0,107
2,221217,Ultrasound,None,1.0,1.0,1.0,142
3,221219,Venogram,None,1.0,1.0,1.0,1
4,221223,EEG,None,1.0,1.0,1.0,31
...,...,...,...,...,...,...,...
91,228127,Family met with Social Worker,None,1.0,1.0,1.0,9
92,228128,Family updated by MD,None,1.0,1.0,1.0,71
93,228129,Family updated by RN,None,1.0,1.0,1.0,168
94,228130,NEOB notified,None,1.0,1.0,1.0,5


In [52]:
# result_df.to_csv('data/MIMIC-III/target_population/sample_stats/procedures_stats.csv', index=False)

## Patient Chronology Analysis
description here